# 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

print(torch.__version__)

2.0.1+cu117


In [2]:
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


## 4.2.1 访问模型参数

## named_parameters()
元组（name,parameter）的生成器


回忆一下上一节中提到的Sequential类与Module类的继承关系。对于Sequential实例中含模型参数的层，我们可以通过Module类的parameters()或者named_parameters方法来访问所有参数（以迭代器的形式返回），后者除了返回参数Tensor外还会返回其名字。下面，访问多层感知机net的所有参数：

In [3]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


可见返回的名字自动加上了层数的索引作为前缀。 我们再来访问net中单层的参数。对于使用Sequential类构造的神经网络，我们可以通过方括号[]来访问网络的任一层。索引0表示隐藏层为Sequential实例最先添加的层。

In [4]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


因为这里是单层的所以没有了层数索引的前缀。另外返回的param的类型为torch.nn.parameter.Parameter，其实这是Tensor的子类，和Tensor不同的是如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里，来看下面这个例子。

In [5]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


<mark>self.para=nn.Parameter(tensor),则此参数进入nn.parameters()中；若不采取nn.Parameter(tensor)而直接用tensor，则此数据成员不会进入该类的.parameters()，进而在net传入optimizer时不能被传入，无法被训练

In [6]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.0743, -0.4650,  0.0546, -0.1508],
        [ 0.2756,  0.0401,  0.2577,  0.0378],
        [ 0.3171,  0.2565, -0.0735, -0.2472]])
None
tensor([[-0.1378, -0.1771, -0.2922, -0.2431],
        [ 0.0713,  0.0916,  0.1511,  0.1258],
        [ 0.3809,  0.4896,  0.8077,  0.6722]])


## 4.2.2 初始化模型参数

In [7]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0048, -0.0072,  0.0022,  0.0012],
        [-0.0046,  0.0048, -0.0062, -0.0105],
        [-0.0080, -0.0145,  0.0046, -0.0179]])
2.weight tensor([[ 0.0090, -0.0095, -0.0089]])


In [8]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 4.2.3 自定义初始化方法

with torch.no_grad():的目的：不求梯度，节约内存

In [9]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()#-->=5保留，<5舍弃成0

TENSOR.uniform_(a,b):将Tensor的每一位都赋成[a,b]内的均匀随机值

In [10]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 5.2091,  0.0000,  0.0000, -0.0000],
        [-5.3733,  7.8664,  0.0000, -0.0000],
        [-0.0000,  0.0000,  6.6323,  0.0000]])
2.weight tensor([[ 0.0000, -0.0000, -6.0652]])


In [11]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


## 4.2.4 共享模型参数

In [12]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [13]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [14]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


tensor(3.)


tensor([[-2., -1.,  0.],
        [ 0.,  1.,  2.]])